In [1]:
#Multilayer Perceptron
# Initialize a network

from random import seed

from random import random

def initialize_network(n_inputs, n_hidden, n_outputs):

    network = list()

    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]

    network.append(hidden_layer)

    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]

    network.append(output_layer)

    return network

seed(1)

network = initialize_network(2, 1, 2)

for layer in network:

    print(layer)


[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [3]:
# Calculate neuron activation for an input

def activate(weights, inputs):

    activation = weights[-1]

    for i in range(len(weights)-1):

        activation += weights[i] * inputs[i]

        return activation

# Transfer neuron activation

from math import exp

def transfer(activation):

    return 1.0 / (1.0 + exp(-activation))


# Forward propagate input to a network output

def forward_propagate(network, row):

    inputs = row

    for layer in network:

        new_inputs = []

        for neuron in layer:

            activation = activate(neuron['weights'], inputs)

            neuron['output'] = transfer(activation)

            new_inputs.append(neuron['output'])

            #print(new_inputs)

        inputs = new_inputs

    return inputs


# test forward propagation

network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],

           [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]

row = [1, 0, None]

output = forward_propagate(network, row)

print(output)


[0.6629970129852887, 0.7253160725279748]


In [6]:
# Transfer Derivative

# derivative = output * (1.0 - output)

# Calculate the Derivative of an neuron output

def transfer_drivative(output):

    return output * (1.0 - output)

# Backpropagate error and store in neurons

def backward_propagate_error(network, expected):

    for i in reversed(range(len(network))):

        layer = network[i]

        errors = list()

        if i != len(network)-1:

            for j in range(len(layer)):

                error = 0.0

                for neuron in network[i + 1]:

                    error += (neuron['weights'][j] * neuron['delta'])

                errors.append(error)

        else:

            for j in range(len(layer)):

                neuron = layer[j]

                errors.append(neuron['output'] - expected[j])

        for j in range(len(layer)):

            neuron = layer[j]

            # The error was on this line due to typo. It was calling 'transfer_drivative' which was not defined
            # The fix is call it correctly as it was defined 'transfer_drivative'
            neuron['delta'] = errors[j] * transfer_drivative(neuron['output'])

# test backpropagation of error

network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],

        [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]

expected = [0, 1]

backward_propagate_error(network, expected)

for layer in network:

    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


In [8]:
# Update Network  weights with error

def update_weights(network, row, l_rate):

    for i in range(len(network)):

        inputs = row[:-1]

        if i != 0:

            inputs = [neuron['output'] for neuron in network[i - 1]]

            for neuron in network[i]:

                for j in range(len(inputs)):

                    neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]

                    neuron['weights'][j] -= l_rate * neuron['delta']

# Train a network for a fixed number of epochs

def train_network(network, train, l_rate, n_epoch, n_outputs):

    for epoch in range(n_epoch):

        sum_error = 0

        for row in train:

            outputs = forward_propagate(network, row)

            expected = [0 for i in range(n_outputs)]

            expected[row[-1]] = 1

            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])

            backward_propagate_error(network, expected)

            update_weights(network, row, l_rate)

        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

# Test training backprop algorithm

seed(1)

dataset = [[2.7810836,2.550537003,0],

    [1.465489372,2.362125076,0],

    [3.396561688,4.400293529,0],

    [1.38807019,1.850220317,0],

    [3.06407232,3.005305973,0],

    [7.627531214,2.759262235,1],

    [5.332441248,2.088626775,1],

    [6.922596716,1.77106367,1],

    [8.675418651,-0.242068655,1],

    [7.673756466,3.508563011,1]]

n_inputs = len(dataset[0]) - 1

n_outputs = len(set([row[-1] for row in dataset]))

network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.5, 20, n_outputs)

for layer in network:

    print(layer)

>epoch=0, lrate=0.500, error=5.488
>epoch=1, lrate=0.500, error=5.337
>epoch=2, lrate=0.500, error=5.269
>epoch=3, lrate=0.500, error=5.245
>epoch=4, lrate=0.500, error=5.239
>epoch=5, lrate=0.500, error=5.240
>epoch=6, lrate=0.500, error=5.241
>epoch=7, lrate=0.500, error=5.243
>epoch=8, lrate=0.500, error=5.245
>epoch=9, lrate=0.500, error=5.246
>epoch=10, lrate=0.500, error=5.247
>epoch=11, lrate=0.500, error=5.247
>epoch=12, lrate=0.500, error=5.247
>epoch=13, lrate=0.500, error=5.248
>epoch=14, lrate=0.500, error=5.248
>epoch=15, lrate=0.500, error=5.248
>epoch=16, lrate=0.500, error=5.248
>epoch=17, lrate=0.500, error=5.248
>epoch=18, lrate=0.500, error=5.248
>epoch=19, lrate=0.500, error=5.248
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.8575212514344961, 'delta': -0.0011567625419637542}, {'weights': [0.2550690257394217, 0.49543508709194095, 0.4494910647887381], 'output': 0.9173465422684997, 'delta': -0.012927763708912984}]
[{'weights': 

In [9]:
#XOR_SLP_MLP

# Activation function

def predict(rows,weights):

    activation=weights[0]

    for i in range(len(rows)-1):

        activation += weights[i+1]*rows[i]

    return 1.0 if activation>=0.0 else 0.0

# Dataset with two inputs
dataset = [[0,0,0],[0,1,1],[1,0,1],[1,1,0]]
# Function for training weights
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            sum_error += error**2
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i+1] = weights[i+1] + l_rate * error * row[i]
        print("> epoch = %d, l_rate = %.3f, error = %.3f" % (epoch, l_rate, sum_error))
        print('Current weights are: ', weights)
    return weights
# Implementing the network
l_rate = 0.1
n_epoch = 5
weights = train_weights(dataset, l_rate, n_epoch)
#print("\ny = %.3f x1 + %.3f x2 + %.3f " % (weights[0],weights[1],weights[2]))

# Code with pre-defined weights
pre_def_weights = [-0.1, 0.1,0.1]
for row in dataset:
    prediction = predict(row,pre_def_weights)
    print("Input = (%d, %d) Expected = %d, Predicted =%d" % (row[0], row[1], row[-1], prediction))


> epoch = 0, l_rate = 0.100, error = 3.000
Current weights are:  [-0.1, -0.1, 0.0]
> epoch = 1, l_rate = 0.100, error = 3.000
Current weights are:  [0.0, -0.1, 0.0]
> epoch = 2, l_rate = 0.100, error = 4.000
Current weights are:  [0.0, -0.1, 0.0]
> epoch = 3, l_rate = 0.100, error = 4.000
Current weights are:  [0.0, -0.1, 0.0]
> epoch = 4, l_rate = 0.100, error = 4.000
Current weights are:  [0.0, -0.1, 0.0]
Input = (0, 0) Expected = 0, Predicted =0
Input = (0, 1) Expected = 1, Predicted =1
Input = (1, 0) Expected = 1, Predicted =1
Input = (1, 1) Expected = 0, Predicted =1
